In [327]:
import numpy as np

In [ ]:
# ============================
# Conversion factors
# ============================

GeVincm     = 5.068e13 # cm^-1 / GeV
GeVinsec    = 1.519e24 # s^-1 / GeV
KinGeV      = 8.62e-14 # GeV / K
GaussinGeV2 = 1.95e-20 # GeV^2 / G

# ============================
# Constants
# ============================

rSun = 8.277 # kpc
sigmaT = 0.6652e-24 # cm^2
me = 5.11e-4 # GeV
alpha = 1/137
e = (4*np.pi*alpha)**.5  # elementary charge in natural units

In [ ]:
# ============================
# Photon field densities
# ============================

def nBB(Eg, T): # GeV^-1 cm^-3
    return GeVincm**3 / np.pi**2 * Eg**2 / (np.exp(Eg/T)-1)

def nISRF(Eg, R, z, T, R0, z0, n0):
    nTOT = GeVincm**3 / np.pi**2 * T**4 * np.pi**4/15
    return n0/nTOT * nBB(Eg, T) * np.exp(-abs(z)/z0) * np.exp(-(R-rSun)/R0)

def nOpt(Eg, R, z):
    return nISRF(Eg, R, z, 5e3*KinGeV, 2.5, 1.5, 0.50e-9)

def nIRw(Eg, R, z):
    return nISRF(Eg, R, z, 400*KinGeV, 3.5, 2, 0.05e-9)

def nIRc(Eg, R, z):
    return nISRF(Eg, R, z, 40*KinGeV,  2, 1, 0.20e-9)

def nCMB(Eg):
    return nBB(Eg, 2.725*KinGeV)

In [ ]:
# ============================
# ICS radiating power
# ============================

def PIC(Eg, Ee, nISRF, *args): # GeV s^-1 GeV^-1

    gamma = Ee / me
    epsilon = Eg / Ee

    def integrand_log(lq, Eg, nISRF, *args):
        q = 10**lq
        Eg0 = Eg / (4 * gamma**2 * (1 - epsilon) * q)

        kernel = (2*q*np.log(q) + q + 1 - 2*q**2 + (epsilon**2 * (1 - q)) / (2 * (1-epsilon))) / q

        return (Eg - Eg0) * nISRF(Eg0, *args) * kernel * (epsilon < 1.0)

    Nlq = 200
    lqmin = np.log10(1/(4*gamma**2))
    lqmax = np.log10(1)

    lqs = np.linspace(lqmin, lqmax, Nlq)
    dlq = (lqmax - lqmin) / Nlq

    prefactor = GeVinsec/GeVincm * 3*sigmaT / (4*gamma**2)

    PIC = 0.0
    for lq in lqs:
        jac = 10**lq * dlq * np.log(10)
        PIC += prefactor * integrand_log(lq, Eg, nISRF, *args) * jac

    return PIC

In [ ]:
# ==============================
# Magnetic field configurations
# ==============================

def BUni(B0): # Gauss
    return B0 

def BGal(R, z, B0, R0, z0):
    return B0 * np.exp(-(R-rSun)/R0) * np.exp(-np.abs(z)/z0)

def BMF1(R, z):
    return BGal(R, z, 4.78e-6, 10., 2)

def BMF2(R, z):
    return BGal(R, z, 5.10e-6, 8.5, 1)

def BMF3(R, z):
    return BGal(R, z, 9.50e-6, 30., 4)

In [ ]:
# ==============================
# Synchrotron radiating power
# ==============================

from scipy.special import kv

def PSyn(nu, Ee, B, *args): # GeV s^-1 Hz^-1

    gamma = Ee/me
    nuc = GeVinsec * 3/(2*np.pi) * gamma**2 * e * B(*args)*GaussinGeV2 / me
    y = nu/nuc

    prefactor = 2 * 3**.5 * e**3 * B(*args)*GaussinGeV2 / me / (4*np.pi)
    kernel = y**2 * (kv(4/3, y)*kv(1/3, y) - 3/5*y * (kv(4/3, y)**2 - kv(1/3, y)**2))

    return prefactor * kernel

In [ ]:
# ==============================
# Gas maps
# ==============================

def ngasGal(R, z, n0, R0, z0): # cm^-3
    return n0 * np.exp(-np.abs(z)/z0) * np.exp(-(R-rSun)/R0)

def nHI(R, z):
    return ngasGal(R, z, 0.5, 10, 0.2)

def nH2(R, z):
    return ngasGal(R, z, 0.15, 10, 0.05)

def nHII(R, z):
    return ngasGal(R, z, 0.03, 10, 1.0)

def nHe(R, z):
    return ngasGal(R, z, 0.1*(0.5+2*0.15+0.03), 10, 0.2)

In [ ]:
# ===============================
# Bremsstrahlung radiating power
# ===============================

def PBrems(Eg, Ee, ngas, *args): # GeV s^-1 GeV^-1
    Eg = np.asarray(Eg)
    Ee = np.asarray(Ee)

    gamma = Ee/me
    epsilon = Eg/Ee

    def dsigmadEg(phi1, phi2):
        prefactor = 3*alpha*sigmaT / (8*np.pi*Eg)
        return prefactor * ((1 + (1-epsilon)**2) * phi1 - 2/3 * (1-epsilon) * phi2)
    
    def phiion(Z):
        return 4 * Z * (Z+1) * (np.log(2*gamma * (1/epsilon-1)) - 1/2)

    phi1HIss, phi2HIss = 45.79, 44.46
    phi1H2ss, phi2H2ss = 2*phi1HIss, 2*phi2HIss
    phi1Hess, phi2Hess = 134.6, 131.4

    if ngas == nHII:
        Pbrems = Eg * ngas(*args) * dsigmadEg(phiion(1), phiion(1))
        Pbrems *= GeVinsec/GeVincm
        return Pbrems
    if ngas == nHI:
        PbremsUR = Eg * ngas(*args) * dsigmadEg(phi1HIss, phi2HIss)
        PbremsLR = Eg * ngas(*args) * dsigmadEg(phiion(1), phiion(1))
    elif ngas == nH2: 
        PbremsUR = Eg * ngas(*args) * dsigmadEg(phi1H2ss, phi2H2ss)
        PbremsLR = Eg * ngas(*args) * dsigmadEg(phiion(2), phiion(2))
    elif ngas == nHe:
        PbremsUR = Eg * ngas(*args) * dsigmadEg(phi1Hess, phi2Hess)
        PbremsLR = Eg * ngas(*args) * dsigmadEg(phiion(2), phiion(2))   

    logg = np.log10(gamma)
    t = (logg - 2.0) / (3.0 - 2.0)
    t = np.clip(t, 0.0, 1.0)
    Pbrems = (1 - t) * PbremsLR + t * PbremsUR

    Pbrems *= GeVinsec/GeVincm

    return Pbrems